# Cirq

In [ ]:
import cirq
from cirq.contrib.svg import SVGCircuit

def get_cirq_compilation(qasm_qc, target):
    if "ibm" in target:
        pass
    elif "rigetti" in target:
        pass
    elif "ionq" in target:
        pass

In [ ]:
from cirq.contrib.qasm_import import circuit_from_qasm
test = open('../MQTbench/mqt/bench/qasm_output/su2random_indep_4.qasm', 'r').read()
ion_circuit = cirq.ion.ConvertToIonGates().convert_circuit(circuit)
print(ion_circuit)

In [ ]:
import cirq_rigetti
c = cirq_rigetti.circuit_transformers.default(circuit=circuit)
print(c)

SVGCircuit(c[0])

In [ ]:
import cirq
import cirq_ionq as ionq
q0 = cirq.LineQubit(0)
device = ionq.IonQAPIDevice([q0])
circuit = cirq.Circuit(device=device)
circuit.append(cirq.H(q0)**0.2) # Non-IonQ-API gate
print(circuit) # will print the circuit converted into IonQ's target gateset

# Pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np

In [ ]:
ibm_native_gates = FakeMontreal().configuration().basis_gates
rigetti_native_gates = ["rx", "rz", "cz"]
ibm_native_gates

dev = qml.device('default.qubit', wires=[0, 1, 2])
qnode = qml.QNode(qfunc, dev)
print(qml.draw(qnode)(0.2, 0.3, 0.4))

compiled_qfunc = qml.compile(basis_set=rigetti_native_gates)(qfunc)
compiled_qnode = qml.QNode(compiled_qfunc, dev)
print(qml.draw(compiled_qnode)(0.2, 0.3, 0.4))

In [ ]:
path_file = "../MQTbench/qasm_output/dj_indep_4.qasm"
with open (path_file, "r") as myfile:
    data = myfile.read()
my_circuit = qml.from_qasm(data)

n=4
dev = qml.device('default.qubit', wires=n)
@qml.qnode(dev)
def circuit():
    my_circuit(wires=tuple(range(n)))
    return [qml.expval(qml.PauliZ(i)) for i in range(n)]

#circuit()
#print(dev._circuit.qasm(formatted=True))

In [ ]:
pennylane_ibm=["RZ", "SX", "PauliX", "CNOT"]
pennylane_rigetti=["RX", "RZ", "CZ"]

In [ ]:
compiled_qfunc1 = qml.compile()(circuit)
compiled_qnode1 = qml.QNode(compiled_qfunc1, dev)
print(qml.draw(compiled_qnode1)())

compiled_qfunc2 = qml.compile(
    basis_set=["CNOT", "RX", "RY", "RZ"],
    num_passes=2
)(circuit)

compiled_qnode2 = qml.QNode(compiled_qfunc2, dev)
print(qml.draw(compiled_qnode2)())

In [ ]:
def qfunc(x, y, z):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RZ(z, wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(z, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(x, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-z, wires=2)
    qml.RX(y, wires=2)
    qml.PauliY(wires=2)
    qml.CY(wires=[1, 2])
    return qml.expval(qml.PauliZ(wires=0))

dev = qml.device('default.qubit', wires=[0, 1, 2])
qnode = qml.QNode(qfunc, dev)
print(qml.draw(qnode)(0.2, 0.3, 0.4))

compiled_qfunc = qml.compile(
    pipeline=[
        qml.transforms.commute_controlled(direction="left"),
        qml.transforms.merge_rotations(atol=1e-6),
        qml.transforms.cancel_inverses
    ],
    basis_set=["CNOT", "RX", "RY", "CZ"],
    num_passes=2
)(qfunc)

compiled_qnode = qml.QNode(compiled_qfunc, dev)
print(qml.draw(compiled_qnode)(0.2, 0.3, 0.4))

In [ ]:
def qfunc():
    qml.Hadamard(wires=0)    
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RZ(-0.4, wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(-0.4, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(-0.4, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-0.4, wires=2)
    qml.RX(1, wires=2)
    qml.PauliY(wires=2)
    qml.CY(wires=[1, 2])
    return [qml.expval(qml.PauliZ(i)) for i in range(2)]

dev = qml.device('default.qubit', wires=[0, 1, 2])
qnode = qml.QNode(qfunc, dev)
print(qml.draw(qnode)())

compiled_qfunc = qml.compile(
    basis_set=["CNOT", "Rdawd", "wdfawdaw", "dawd"],
    num_passes=20
)(qfunc)

compiled_qnode = qml.QNode(compiled_qfunc, dev)
print(qml.draw(compiled_qnode)())

# Machine Learning Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

# Feature Vector Generation

In [1]:
from mqt.bench import benchmark_generator
import matplotlib.pyplot as plt
from evaluator.qiskit_plugin import *
from evaluator.pytket_plugin import *
import numpy as np

In [2]:
from evaluator.utils import get_openqasm_gates
def dict_to_featurevector(gate_dict):
    openqasm_gates_list = get_openqasm_gates()
    res_dct = {openqasm_gates_list[i] for i in range(0, len(openqasm_gates_list))}
    res_dct = dict.fromkeys(res_dct, 0)
    for key, val in dict(gate_dict).items():
        if not key in res_dct:
            print(key, "gate not found in openQASM 2.0 gateset")
        else:
            res_dct[key] = val
    return res_dct

In [3]:
benchmarks = ["dj", "ghz", "wstate", "graphstate", "ae", "qpeexact", "twolocalrandom", "qft", "qftentangled", "su2random" ]
res = []
for benchmark in benchmarks:
    for num_qubits in range(3,50,5):
        print(benchmark, num_qubits)
        qc = benchmark_generator.get_one_benchmark(benchmark, 1, num_qubits)
        qasm_qc = qc.qasm()
        ops_list = qc.count_ops()
        try:
            best_arch = np.argmin(get_tket_scores(qasm_qc)+get_qiskit_scores(qasm_qc))
            res.append((ops_list, best_arch, num_qubits))
        except Exception as e:
            print("fail: ", e)

dj 3


/Users/nils/miniforge3/envs/tf3/lib/python3.9/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/nils/miniforge3/envs/tf3/lib/python3.9/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


[33.058312020460356, 33.058312020460356, 0.2, 538.4342897626639, 0.0]
dj 8
[196.35130434782607, 159.3879283887468, 0.2, 1610.3436060928088, 0.0]
dj 13
[609.8513554987211, 313.1450639386189, 100000, 4194.119730782855, 100000]
dj 18
[790.036726342711, 456.36705882352936, 100000, 7531.70386114063, 100000]
dj 23
[953.3297186700767, 698.4911508951405, 100000, 8619.199433227062, 100000]
dj 28
[1228.5121227621482, 100000, 100000, 11488.487424725468, 100000]
dj 33
[1405.5188746803067, 100000, 100000, 13507.97024442083, 100000]
dj 38
[1440.0300767263425, 100000, 100000, 14868.933758413035, 100000]
dj 43


KeyboardInterrupt: 

In [ ]:
training_data = []
for elem in res:
    tmp=dict_to_featurevector(elem[0])
    tmp["num_qubits"]= elem[2]
    training_data.append((list(tmp.values()), elem[1]))
print(training_data)

In [ ]:
best_machine_index = []
for elem in res:
    best_machine_index.append(elem[1])
plt.hist(best_machine_index)
plt.xticks([0,1,2,3,4,5]);

In [ ]:
x_train, y_train = zip(*training_data)
with open('x_train.csv', 'w') as FOUT:
    np.savetxt(FOUT, x_train)
with open('y_train.csv', 'w') as FOUT:
    np.savetxt(FOUT, y_train)

In [ ]:
print(y_train)

In [ ]:
from numpy import genfromtxt
X = genfromtxt('x_train.csv')
y = genfromtxt('y_train.csv')
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(500, activation='relu', input_dim=43))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100)

In [ ]:
pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))    

# TKET IonQ Custom Rebase Pass

In [ ]:
import pytket
from pytket import OpType, circuit, _tket, Circuit
from pytket.passes import RebaseCustom

In [ ]:
gates = {OpType.Rz, OpType.Ry, OpType.CY, OpType.ZZPhase}
cx_in_cy = Circuit(2)
cx_in_cy.Rz(0.5, 1).CY(0, 1).Rz(-0.5, 1)
def tk1_to_rzry(a, b, c):
    circ = Circuit(1)
    circ.Rz(c + 0.5, 0).Ry(b, 0).Rz(a - 0.5, 0)
    return circ

custom = RebaseCustom(gates, cx_in_cy, tk1_to_rzry)

circ = Circuit(3)
circ.X(0).CX(0, 1).Ry(0.2, 1)
circ.add_gate(OpType.ZZPhase, -0.83, [2, 1]).Rx(0.6, 2)

custom.apply(circ)

print(circ.get_commands())

In [ ]:
ionq_gateset = {OpType.Rz, OpType.Ry, OpType.XXPhase}
cx_in_ionq = _tket.circuit._library._CX_using_XXPhase_0()
def tk1_to_rzry(a, b, c):
    circ = Circuit(1)
    circ.Rz(c + 0.5, 0).Ry(b, 0).Rz(a - 0.5, 0)
    return circ
ionq_rebase = RebaseCustom(ionq_gateset, cx_in_ionq, tk1_to_rzry)

In [ ]:
circ = Circuit(3)
circ.X(0).CX(0, 1).Ry(0.2, 1)
circ.add_gate(OpType.ZZPhase, -0.83, [2, 1]).Rx(0.6, 2)

ionq_rebase.apply(circ)
print(circ.get_commands())

In [ ]:
from pytket._tket.circuit._library._CX_using_XXPhase_0()

In [ ]:
_library._CX_using_XXPhase_0()

# Testing/Debugging

In [ ]:
from evaluator import utils
from pytket.circuit.display import render_circuit_jupyter

In [ ]:
qc_raw = benchmark_generator.get_one_benchmark("vqe", 1, 5)
qasm_qc = qc_raw.qasm()
qc = qasm.circuit_from_qasm_str(qasm_qc)

In [ ]:
from pytket.extensions.pyquil import ForestStateBackend
backend = ForestStateBackend()
rigetti_arch = pytket.architecture.Architecture(get_cmap_rigetti_m1(10))
render_circuit_jupyter(qc)
backend.rebase_pass().apply(qc)
PlacementPass(LinePlacement(rigetti_arch)).apply(qc)
RoutingPass(rigetti_arch).apply(qc)
backend.rebase_pass().apply(qc)
render_circuit_jupyter(qc)

In [ ]:
render_circuit_jupyter(qc)
ionq_rebase = ge()
ionq_rebase.apply(qc)
render_circuit_jupyter(qc)

In [ ]:
qc = benchmark_generator.get_one_benchmark("dj", 1, 5)

In [ ]:
from qiskit import transpile
oqc_gates = ['rz', 'ecr', 'sx', 'x']
qc_oqc = transpile(qc, basis_gates=oqc_gates)
oqc = get_oqc_lucy()
#print(qc_oqc.qasm())
score_oqc = calc_score_from_str(qc_oqc.qasm(), oqc)

In [ ]:
from qiskit.test.mock import (
    FakeBogota,
    FakeCasablanca,
    FakeGuadalupe,
    FakeMontreal,
    FakeManhattan,
)

In [ ]:
FakeMontreal().configuration().configuration().coupling_map